In [ ]:
project_path = "/home/jupyter"
import os
import sys

sys.path.append(project_path)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from google.cloud import bigquery

from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t


client = bigquery.Client()

In [ ]:
# Summarise the data by mcg
UK_spending_by_country = '''SELECT time_period_value, destination_country, spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter'
and time_period_value != '2025Q1'  
and merchant_channel = 'Online'
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM'
and destination_country != 'UNITED KINGDOM'
and mcg = 'All'
GROUP BY destination_country, time_period_value, spend
ORDER BY time_period_value, spend DESC'''

yearly_country_df = bq.read_bq_table_sql(client, UK_spending_by_country)

yearly_country_df['year'] = yearly_country_df['time_period_value'].str[:4]

yearly_country_df.head()

In [ ]:
yearly_country_df.to_csv('yearly_country_df.csv')
df = pd.read_csv('yearly_country_df.csv')
latest_country_df = df[df['year'] == 2024][['destination_country', 'spend']]
latest_country_df.to_csv('2024_country.csv', index=False)

In [ ]:
total_spend_per_country = latest_country_df.groupby('destination_country')['spend'].sum()
print(total_spend_per_country)

In [ ]:
total_spend_per_country_perc = (total_spend_per_country / total_spend_per_country.sum()) * 100
print(total_spend_per_country_perc)

In [ ]:
# Group by 'mcg' and sum the 'spend'
total_spend_per_country = latest_country_df.groupby('destination_country')['spend'].sum()

# Get the top 10 'mcg' categories by spend
top_10_country = total_spend_per_country.nlargest(10)

# Check if the result is not empty
if not top_10_country.empty:
    # Calculate the percentage of total spend for these top 10
    top_10_country_percentage = (top_10_country / top_10_country.sum()) * 100

    # Plot the bar chart
    ax = top_10_country_percentage.plot(kind='bar', color='skyblue')

    # Customize the plot
    plt.ylabel('Percentage of Online Spend')
    plt.title('Top 10 Countries by percentage of total online spend, 2024')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print("No data available to plot.")